In [4]:
import string
import json
import pandas as pd
import re

raw_df = pd.read_excel("Chat Conv Part A.xlsx")
df_grouped = raw_df.groupby('chat_id')


def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data



def save_json_file(output_path, data):
   with open(output_path, 'w') as file:
        json.dump(data, file, indent=4)

def split_json_file(json_data):
    
    json_data1, json_data2 = {}, {}
    mid_index = len(json_data) // 2
    
    for key, value in json_data.items():
        if len(json_data1) < mid_index:
            json_data1[key] = value
        else:
            json_data2[key] = value

    return json_data1, json_data2



In [5]:
json_data = {}

not_enough = {}
counter = 0
# Loop through grouped DataFrame (replace with your grouped DataFrame name)
for group_name, group_df in df_grouped: 
    counter += 1
    if (counter % 500 == 0): 
        print(counter)
        
    if ((group_df.iloc[0]['is_english']) == False):
        continue
        
    
    
    
    welcome_message = 0
    
    # Reset the index to start from 0
    group_df = group_df.reset_index(drop=True)
    dialog_dict = {"number_of_turns": 0,
                            "annotator_id": None,
                            "dialog": []}

    # Add the welcome message if it exists
    if str(group_df.iloc[0]['welcome_message']) != "nan":
        dialog_dict["number_of_turns"] += 1
        welcome_message = 1
        dialog_dict["dialog"].append({
            "turn_num": 0,
            "original_question": group_df.iloc[0]['welcome_message'],
            "answer": "",
        })

    turns_counter = 0

    # Loop through the DataFrame by 2 rows at a time
    for index in range(0, len(group_df), 2):
        
         # Increment the number of turns
        dialog_dict["number_of_turns"] += 1
    
        # Get the current row
        row1 = group_df.iloc[index]
       
        if index + 1 < len(group_df):
            row2 = group_df.iloc[index + 1]
        else:
            row2 = None  # If the number of rows is odd, the last pair will have None for the second row
    
        # Add the two rows to the JSON
        dialog_dict["dialog"].append({
        "turn_num": int(index / 2) + welcome_message,
        "original_question": str(row1['message']),
        "answer": str(row2['message']) if row2 is not None else "None",
        })

        if index + welcome_message> 0:
            turns_counter += 1
            dialog_dict[int(index / 2) + welcome_message] = {
                'requires_rewrite': None,
                'enough_context': None
            }

    if turns_counter > 0:
        json_data[group_name] = dialog_dict
    else:
        not_enough[group_name] = dialog_dict
        


500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500


In [53]:
json_data

{'00630978-c608-4bad-ac69-f8af2a98f3b9': {'number_of_turns': 3,
  'annotator_id': None,
  'dialog': [{'turn_num': 0,
    'original_question': 'Hi tyler: I am Chantal and living in Puerto Vallarta Mexico. Do you ship to Mexico? If yes, approx how much would that cost or do you have dealers in Mexico?\n',
    'answer': 'We only ship across CONUS (Continental United States) and currently cannot ship to Mexico.'},
   {'turn_num': 1,
    'original_question': 'how about Germany?',
    'answer': 'Addresses to which we ship: We ship across CONUS and currently cannot ship to Hawaii, Puerto Rico, and Alaska.'},
   {'turn_num': 2,
    'original_question': "that's a no?",
    'answer': "As the OneAgent, I'm trained to only answer relevant questions."}],
  1: {'requires_rewrite': None, 'enough_context': None},
  2: {'requires_rewrite': None, 'enough_context': None}},
 '0072b7b6-50d8-44d2-8aa2-cfa8df9b9e35': {'number_of_turns': 2,
  'annotator_id': None,
  'dialog': [{'turn_num': 0,
    'original_qu

In [6]:
save_json_file("yael_a1.json", json_data)